In [30]:
import pandas as pd
import numpy as np
import fastf1 as f1

f1.Cache.enable_cache('fastf1cache')

session = f1.get_session(2024, 'Abu Dhabi', 'R')
session.load()
session.results
# events = f1.get_event_schedule(2024)
# events
# events.get('EventName')

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']


,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Laps
4,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,1.0,1,1.0,NaT,NaT,NaT,0 days 01:26:33.291000,Finished,25.0,58.0
55,55,C SAINZ,SAI,sainz,Ferrari,E80020,ferrari,Carlos,Sainz,Carlos Sainz,...,2.0,2,3.0,NaT,NaT,NaT,0 days 00:00:05.832000,Finished,18.0,58.0
16,16,C LECLERC,LEC,leclerc,Ferrari,E80020,ferrari,Charles,Leclerc,Charles Leclerc,...,3.0,3,19.0,NaT,NaT,NaT,0 days 00:00:31.928000,Finished,15.0,58.0
44,44,L HAMILTON,HAM,hamilton,Mercedes,27F4D2,mercedes,Lewis,Hamilton,Lewis Hamilton,...,4.0,4,16.0,NaT,NaT,NaT,0 days 00:00:36.483000,Finished,12.0,58.0
63,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,5.0,5,6.0,NaT,NaT,NaT,0 days 00:00:37.538000,Finished,10.0,58.0
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,6.0,6,4.0,NaT,NaT,NaT,0 days 00:00:49.847000,Finished,8.0,58.0
10,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,...,7.0,7,5.0,NaT,NaT,NaT,0 days 00:01:12.560000,Finished,6.0,58.0
27,27,N HULKENBERG,HUL,hulkenberg,Haas F1 Team,B6BABD,haas,Nico,Hulkenberg,Nico Hulkenberg,...,8.0,8,7.0,NaT,NaT,NaT,0 days 00:01:15.554000,Finished,4.0,58.0
14,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,9.0,9,8.0,NaT,NaT,NaT,0 days 00:01:22.373000,Finished,2.0,58.0
81,81,O PIASTRI,PIA,piastri,McLaren,FF8000,mclaren,Oscar,Piastri,Oscar Piastri,...,10.0,10,2.0,NaT,NaT,NaT,0 days 00:01:23.821000,Finished,1.0,58.0
